# Databases - les 10 - 8/05

installeer packages mysql-connector-python

In [1]:
! pip install mysql-connector-python


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Connectie databank

XAMPP --> username: 'root' en paswoord: ''

In [4]:
import mysql.connector

db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="movies_db"
)

cursor = db.cursor()

# doe je iets

cursor.close()
db.close()

## CREATE

3 verschillende manieren:
- single record
- meerdere records met loop
- meerdere records met executemany

In [7]:
import mysql.connector

db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="movies_db"
)

cursor = db.cursor()

query = "INSERT INTO actors(name) VALUES ('Cameron Diaz');" # als je kopieert vanuit PhpMyAdmin verwijder je best alle 'rare' quotes
cursor.execute(query)

db.commit()
db.close()


In [12]:
import mysql.connector

db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="movies_db"
)

cursor = db.cursor()

query = "INSERT INTO actors(name) VALUES (%s);" 
data = [["Nicolas Cage"],["Johnny Depp"],["Scarlet Johanson"],["Amber Heard"]]
for person in data:
    cursor.execute(query,person)

db.commit()
db.close()

In [14]:
import mysql.connector

db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="movies_db"
)

cursor = db.cursor()

query = "INSERT INTO actors(name) VALUES (%s);" 
data = [["Matt Damon"],["Shelly Duval"],["Matthias Schoenaerts"],["Veerle Baetens"]]
cursor.executemany(query,data)

db.commit()
db.close()

## READ

In [20]:
import mysql.connector

db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="movies_db"
)

cursor = db.cursor()

query = "select * from actors;"
cursor.execute(query)

results = cursor.fetchall()

for person in results:
    print(person)
    print(person[0], person[1])

db.close()

(1, 'Michelle Yeoh')
1 Michelle Yeoh
(2, 'Stephan James')
2 Stephan James
(3, 'Jamie Lee Curtis')
3 Jamie Lee Curtis
(4, 'Tom Cruise')
4 Tom Cruise
(5, 'Brad Pitt')
5 Brad Pitt
(6, 'Cameron Diaz')
6 Cameron Diaz
(7, 'Nicolas Cage')
7 Nicolas Cage
(8, 'Johnny Depp')
8 Johnny Depp
(9, 'Scarlet Johanson')
9 Scarlet Johanson
(10, 'Amber Heard')
10 Amber Heard
(11, 'Matt Damon')
11 Matt Damon
(12, 'Shelly Duval')
12 Shelly Duval
(13, 'Matthias Schoenaerts')
13 Matthias Schoenaerts
(14, 'Veerle Baetens')
14 Veerle Baetens


In [21]:
import mysql.connector

db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="movies_db"
)

cursor = db.cursor()

query = "select * from actors;"
cursor.execute(query)

columns = [desc[0] for desc in cursor.description]

results = cursor.fetchall()

for person in results:
    print(person[columns.index("name")])

db.close()

Michelle Yeoh
Stephan James
Jamie Lee Curtis
Tom Cruise
Brad Pitt
Cameron Diaz
Nicolas Cage
Johnny Depp
Scarlet Johanson
Amber Heard
Matt Damon
Shelly Duval
Matthias Schoenaerts
Veerle Baetens


In [23]:
import mysql.connector

db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="movies_db"
)

cursor = db.cursor()

query = """select actors.name, movies.title 
            from movie_actor 
                inner join actors on actors.id = movie_actor.actor_id 
                inner join movies on movies.id = movie_actor.movie_id;"""
cursor.execute(query)

results = cursor.fetchall()

print(results)

for row in results:
    print(row)

db.close()

[('Michelle Yeoh', 'Everything Everywhere All at Once'), ('Stephan James', 'Everything Everywhere All at Once'), ('Jamie Lee Curtis', 'Everything Everywhere All at Once'), ('Tom Cruise', 'Top Gun: Maverick')]
('Michelle Yeoh', 'Everything Everywhere All at Once')
('Stephan James', 'Everything Everywhere All at Once')
('Jamie Lee Curtis', 'Everything Everywhere All at Once')
('Tom Cruise', 'Top Gun: Maverick')


## DELETE

Vergeet geen 'Dummy Movie' toe te voegen aan de tabel

In [25]:

import mysql.connector

db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="movies_db"
)

cursor = db.cursor()
# delete a specific movie
delete_query = "DELETE FROM movies WHERE title = %s"
delete_data = ("Dummy Movie",)
cursor.execute(delete_query, delete_data)

# commit changes
db.commit()

# check if movie was deleted
select_query = "SELECT * FROM movies WHERE title = %s"
select_data = ("Dummy Movie",)
cursor.execute(select_query, select_data)

if cursor.rowcount == 0:
    print("Movie was deleted successfully")
else:
    print("Movie was not deleted")

Movie was deleted successfully


## Scrape data into database

https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films


In [48]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

url = "https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films"
response = requests.get(url)
soup = BeautifulSoup(response.text)

tbody = soup.find("tbody")
trs = tbody.find_all("tr")

for tr in trs:
    td = tr.find_all("td")
    if len(td) == 4:
        title = td[0].get_text().replace("'","")
        year = td[1].get_text()
        awards = td[2].get_text()
        nom = td[3].get_text()

        # print(title, year, awards, nom)
        db = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="movies_db"
        )
        cursor = db.cursor()
        query = "SELECT * FROM movies WHERE title like '" + title + "';"
        cursor.execute(query)
        rows = cursor.fetchall()

        if len(rows) == 0:
            query = "INSERT INTO movies(title) VALUES ('" + title + "')"
            cursor.execute(query)
            db.commit()
            id = cursor.lastrowid

            query = "INSERT INTO awards(awardshow_id, year, movie_id, awards, nominations) VALUES (%s,%s,%s,%s,%s);"
            data = (1,year, id, awards, nom)
            cursor.execute(query,data)
            db.commit()

        db.close()
        

